In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=cf4b6312918fa357bc32e6e921351b550817468f3bb2019e5b6e780851b823a5
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark for Prediction')\
        .getOrCreate()

In [15]:
df=spark.read.csv('flights-larger.csv', sep =',', header=True, inferSchema=True, nullValue='NA')

In [4]:
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| null|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [5]:
df.dtypes

[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('flight', 'int'),
 ('org', 'string'),
 ('mile', 'int'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('delay', 'int')]

In [6]:
df.take(5)

[Row(mon=10, dom=10, dow=1, carrier='OO', flight=5836, org='ORD', mile=157, depart=8.18, duration=51, delay=27),
 Row(mon=1, dom=4, dow=1, carrier='OO', flight=5866, org='ORD', mile=466, depart=15.5, duration=102, delay=None),
 Row(mon=11, dom=22, dow=1, carrier='OO', flight=6016, org='ORD', mile=738, depart=7.17, duration=127, delay=-19),
 Row(mon=2, dom=14, dow=5, carrier='B6', flight=199, org='JFK', mile=2248, depart=21.17, duration=365, delay=60),
 Row(mon=5, dom=25, dow=3, carrier='WN', flight=1675, org='SJC', mile=386, depart=12.92, duration=85, delay=22)]

In [7]:
df.describe()

DataFrame[summary: string, mon: string, dom: string, dow: string, carrier: string, flight: string, org: string, mile: string, depart: string, duration: string, delay: string]

In [16]:
df.count()

275000

In [9]:
df.distinct()

DataFrame[mon: int, dom: int, dow: int, carrier: string, flight: int, org: string, mile: int, depart: double, duration: int, delay: int]

In [10]:
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| null|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [17]:
df=df.drop('flight')

In [12]:
import numpy as np
import pandas as pd

In [18]:
df =df.dropna()

In [19]:
df.count()

258289

In [20]:
df.show(5)

+---+---+---+-------+---+----+------+--------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|
+---+---+---+-------+---+----+------+--------+-----+
| 10| 10|  1|     OO|ORD| 157|  8.18|      51|   27|
| 11| 22|  1|     OO|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|SJC| 386| 12.92|      85|   22|
|  3| 28|  1|     B6|LGA|1076| 13.33|     182|   70|
+---+---+---+-------+---+----+------+--------+-----+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import round
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [22]:
from pyspark.ml.stat import Correlation

In [23]:
df=StringIndexer(inputCol='carrier', outputCol='carrier_int').fit(df).transform(df)

In [24]:
df=StringIndexer(inputCol='org', outputCol='org_int').fit(df).transform(df)

In [27]:
df = df.withColumn('label', (df.delay > 15).cast('integer'))

In [29]:
df.show(5)

+---+---+---+-------+---+----+------+--------+-----+-----------+-------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|carrier_int|org_int|label|
+---+---+---+-------+---+----+------+--------+-----+-----------+-------+-----+
| 10| 10|  1|     OO|ORD| 157|  8.18|      51|   27|        2.0|    0.0|    1|
| 11| 22|  1|     OO|ORD| 738|  7.17|     127|  -19|        2.0|    0.0|    0|
|  2| 14|  5|     B6|JFK|2248| 21.17|     365|   60|        4.0|    2.0|    1|
|  5| 25|  3|     WN|SJC| 386| 12.92|      85|   22|        3.0|    5.0|    1|
|  3| 28|  1|     B6|LGA|1076| 13.33|     182|   70|        4.0|    3.0|    1|
+---+---+---+-------+---+----+------+--------+-----+-----------+-------+-----+
only showing top 5 rows



In [30]:
assembler = VectorAssembler(inputCols=['mon', 'dom', 'dow',
'carrier_int', 'org_int', 'mile', 'depart', 'duration'], outputCol='features')
# Consolidate predictor columns
df_transform = assembler.transform(df)
# Check the resulting column
# df_transform.select('features', 'delay').show(5, truncate=False)
df_transform.select('features', 'delay')

DataFrame[features: vector, delay: int]

In [35]:
Correlation.corr(df_transform, 'features').collect()[0][0]

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


DenseMatrix(8, 8, [1.0, 0.0113, -0.0182, 0.0285, 0.0092, -0.0135, -0.0143, -0.0105, ..., -0.0105, 0.0002, 0.0104, -0.1689, -0.0845, 0.9809, -0.0419, 1.0], False)

In [36]:
df_train, df_test = df_transform.randomSplit([0.8, 0.2], seed=52)

In [37]:
df_train.show(5)

+---+---+---+-------+---+----+------+--------+-----+-----------+-------+-----+--------------------+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|carrier_int|org_int|label|            features|
+---+---+---+-------+---+----+------+--------+-----+-----------+-------+-----+--------------------+
|  0|  1|  2|     AA|JFK| 740| 15.75|     175|   17|        1.0|    2.0|    1|[0.0,1.0,2.0,1.0,...|
|  0|  1|  2|     AA|JFK| 944| 15.75|     185|   16|        1.0|    2.0|    1|[0.0,1.0,2.0,1.0,...|
|  0|  1|  2|     AA|JFK|1391| 14.92|     245|    6|        1.0|    2.0|    0|[0.0,1.0,2.0,1.0,...|
|  0|  1|  2|     AA|JFK|1597|  6.58|     230|   50|        1.0|    2.0|    1|[0.0,1.0,2.0,1.0,...|
|  0|  1|  2|     AA|JFK|1597|  13.5|     225|    1|        1.0|    2.0|    0|[0.0,1.0,2.0,1.0,...|
+---+---+---+-------+---+----+------+--------+-----+-----------+-------+-----+--------------------+
only showing top 5 rows



In [38]:
training_ratio = df_train.count() / df.count()
print(training_ratio)

0.8013349387701373


In [39]:
model = DecisionTreeClassifier()
model_fit = model.fit(df_train)
prediction = model_fit.transform(df_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|0    |1.0       |[0.3776145354280482,0.6223854645719518]|
|0    |1.0       |[0.3776145354280482,0.6223854645719518]|
|1    |1.0       |[0.3776145354280482,0.6223854645719518]|
|0    |1.0       |[0.3776145354280482,0.6223854645719518]|
|1    |1.0       |[0.3776145354280482,0.6223854645719518]|
+-----+----------+---------------------------------------+
only showing top 5 rows



In [40]:
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 7939|
|    0|       0.0|14556|
|    1|       1.0|17840|
|    0|       1.0|10978|
+-----+----------+-----+

0.6313409857151209


In [42]:
df_updated = df.drop('depart')
df_updated.show(5)

+---+---+---+-------+---+----+--------+-----+-----------+-------+-----+
|mon|dom|dow|carrier|org|mile|duration|delay|carrier_int|org_int|label|
+---+---+---+-------+---+----+--------+-----+-----------+-------+-----+
| 10| 10|  1|     OO|ORD| 157|      51|   27|        2.0|    0.0|    1|
| 11| 22|  1|     OO|ORD| 738|     127|  -19|        2.0|    0.0|    0|
|  2| 14|  5|     B6|JFK|2248|     365|   60|        4.0|    2.0|    1|
|  5| 25|  3|     WN|SJC| 386|      85|   22|        3.0|    5.0|    1|
|  3| 28|  1|     B6|LGA|1076|     182|   70|        4.0|    3.0|    1|
+---+---+---+-------+---+----+--------+-----+-----------+-------+-----+
only showing top 5 rows



In [44]:
assembler = VectorAssembler(inputCols=['mon', 'dom', 'dow',
'carrier_int', 'org_int', 'mile', 'duration'], outputCol='features')
# Consolidate predictor columns
df_transform1 = assembler.transform(df_updated)
# Check the resulting column
df_transform1.select('features', 'delay').show(5, truncate=False)
#df_transform1.select('features', 'delay')

+-----------------------------------+-----+
|features                           |delay|
+-----------------------------------+-----+
|[10.0,10.0,1.0,2.0,0.0,157.0,51.0] |27   |
|[11.0,22.0,1.0,2.0,0.0,738.0,127.0]|-19  |
|[2.0,14.0,5.0,4.0,2.0,2248.0,365.0]|60   |
|[5.0,25.0,3.0,3.0,5.0,386.0,85.0]  |22   |
|[3.0,28.0,1.0,4.0,3.0,1076.0,182.0]|70   |
+-----------------------------------+-----+
only showing top 5 rows



In [45]:
df_updated_train, df_updated_test = df_transform1.randomSplit([0.8, 0.2], seed=42)

In [49]:
model_new = DecisionTreeClassifier()
model_new_fit = model_new.fit(df_updated_train)
prediction = model_new_fit.transform(df_updated_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|0    |1.0       |[0.37475178713264495,0.625248212867355]|
|0    |1.0       |[0.37475178713264495,0.625248212867355]|
|0    |1.0       |[0.37475178713264495,0.625248212867355]|
|0    |1.0       |[0.37475178713264495,0.625248212867355]|
|0    |1.0       |[0.37475178713264495,0.625248212867355]|
+-----+----------+---------------------------------------+
only showing top 5 rows



In [50]:
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 7272|
|    0|       0.0|13375|
|    1|       1.0|18359|
|    0|       1.0|12720|
+-----+----------+-----+

0.6135019139310984


In [51]:
LR = LogisticRegression()
model = LR.fit(df_train)
prediction = model.transform(df_test)
prediction.groupby('label','prediction').count().show(5)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 9807|
|    0|       0.0|15204|
|    1|       1.0|15972|
|    0|       1.0|10330|
+-----+----------+-----+



In [ ]:
df.filter('delay > 15').count()

129062

In [ ]:
df.filter('delay <= 15').count()

129227

In [ ]:
df.select('mile').show()

+----+
|mile|
+----+
| 157|
| 738|
|2248|
| 386|
|1076|
| 740|
| 679|
| 214|
|1197|
|1182|
|1090|
|1028|
| 733|
| 733|
| 292|
| 612|
| 301|
| 414|
|1846|
| 337|
+----+
only showing top 20 rows

